# Dosimetry Assignment
### Lav en Dosimeter klasse der kan loade, holde og analysere dosimetri data. Sammenlign dosimetri data for virksomhederne

Nedenunder har vi lavet klassen Dosemetry, der under oprettelse gives ét Person ID, fra én person.

Metoden load_patient_log() gør det muligt at indlæse og formatere Dosimetri data ud fra en .txt fil.

Metoden load_fyraften() indlæser data fra Fyraften.csv filen og formaterer disse.

limit_patient_data_by_fyraften() fjerner alle data efter fyraften for person.

calculate_total_leq() udregner Total L_eq og calculate_lex8() udregner L_ex,8.

Til sidst samler run_full_analysis() ovenstående metoder og benytter disse til at udregne L_eq, L_ex,8 og total arbejdstid for én person.

In [38]:
import pandas as pd
import numpy as np

# Klassen modtager én patient_id og behandler data på tværs a filer for denne.
class Dosemetry:
    def __init__(self, patient_id):
        self.patient_id = str(patient_id)
        self.patient_df = pd.DataFrame()
        self.firma = None
        self.fyraften_df = pd.DataFrame()
        self.filtered_df = pd.DataFrame()
        self.total_hours_worked = None
        self.Total_Leq = None
        self.Lex8 = None

    #Indlæser og konverterer dosimetri data fra PATIENT_ID.txt fil til Pandas DataFrame
    def load_patient_log(self):
        file_path = f"DosimetriData/{self.patient_id}.txt"

        #Skip øverste 11 linjer og sorter data ud fra mellemrum i teksten.
        df = pd.read_csv(file_path, sep=r"\s+", header=None, skiprows=11, engine="python")
        df.columns = ['Datapoint', 'Flags', 'Date', 'Time', 'Leq', 'CPEAK', 'Extra']

        #Kombiner Dato og Tid til én værdi og gem denne
        df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d.%m.%Y %H:%M:%S', errors='coerce')
        # Formater Leq decimal fra , til .
        df['Leq'] = pd.to_numeric(df['Leq'].str.replace(',', '.', regex=False), errors='coerce')
        # Fjern rækker hvor Datetime eller Leq ikke eksisterer
        df = df.dropna(subset=['Datetime', 'Leq'])

        self.patient_df = df
        return df

    #Indlæser og konverterer data fra Fyraften.csv til Pandas Dataframe
    def load_fyraften(self):
        df = pd.read_csv("DosimetriData/Fyraften.csv", sep=';')
        df.columns = [col.strip().lower() for col in df.columns]

        if 'patientid' not in df.columns:
            if 'personid' in df.columns:
                df = df.rename(columns={'personid': 'patientid'})

        self.fyraften_df = df
        return df

    def get_firma(self):
        df = pd.read_csv("DosimetriData/Noeglefil.csv", sep=';')
        df.columns = [col.strip().lower() for col in df.columns]
        result = df.loc[df["personid"] == int(self.patient_id), "virksomhedsnavn"]
        if not result.empty:
            self.firma = result.iloc[0]
        else:
            self.firma = None

    # Fjerner alle målinger efter fyraften
    def limit_patient_data_by_fyraften(self):
        if self.fyraften_df.empty or self.patient_df.empty:
            self.filtered_df = self.patient_df
            self.total_hours_worked = None
            return self.patient_df

        row = self.fyraften_df[self.fyraften_df['patientid'].astype(str) == self.patient_id]

        if row.empty:
            self.filtered_df = self.patient_df
            self.total_hours_worked = None
            return self.patient_df

        fyraften_str = str(row.iloc[0]['fyraften'])

        if not self.patient_df.empty:
            first_date = self.patient_df.iloc[0]['Date']
            fyraften_datetime = pd.to_datetime(f"{first_date} {fyraften_str}", format='%d.%m.%Y %H.%M', errors='coerce')
        else:
            fyraften_datetime = pd.NaT

        filtered_df = self.patient_df[self.patient_df['Datetime'] < fyraften_datetime]
        self.filtered_df = filtered_df

        if not filtered_df.empty:
            start_time = filtered_df.iloc[0]['Datetime']
            end_time = filtered_df.iloc[-1]['Datetime']
            self.total_hours_worked = (end_time - start_time).total_seconds() / 3600.0
        else:
            self.total_hours_worked = None

        return filtered_df

    # Udregn total L_eq
    def calculate_total_leq(self):
        df = self.filtered_df

        if df.empty or 'Leq' not in df:
            return None

        leq_values = df['Leq'].dropna().values
        if len(leq_values) == 0:
            return None

        total_leq = 10 * np.log10(np.mean(10 ** (leq_values / 10)))
        return round(total_leq, 3)

    # Udregn L_ex,8
    def calculate_lex8(self):
        leq_t = self.calculate_total_leq()
        T = self.total_hours_worked

        if leq_t is None or T is None or T <= 0:
            return None

        lex8 = leq_t + 10 * np.log10(T / 8)
        return round(lex8, 3)

    def run_full_analysis(self):
        self.load_patient_log()
        self.load_fyraften()
        self.limit_patient_data_by_fyraften()

        self.Total_Leq = self.calculate_total_leq()
        self.Lex8 = self.calculate_lex8()

        if self.total_hours_worked is not None:
            self.total_hours_worked = round(self.total_hours_worked, 3)

Nedenunder vises eksempel hvor Total Leq, Lex,8 og total arbejdstid udregnes for én Person ID (129).

In [40]:
P_129 = Dosemetry(129)
P_129.run_full_analysis()
total_leq = P_129.Total_Leq
lex8 = P_129.Lex8
hours = P_129.total_hours_worked
print(f"Total Leq: {total_leq:.3f}")
print(f"Lex,8: {lex8:.3f}")
print(f"Total hours worked: {hours:.3f}")

Total Leq: 67.365
Lex,8: 67.948
Total hours worked: 9.149


Vi kan nu udregne værdierne for alle PersonID, ved at samle disse i en liste og så loope igennem dem.

In [41]:
# Script to run analysis for all patients
patient_ids = [129,131,133,142,188,228,242,254,268,386,444,453,480,495,501,537,541,681,689,690,693,711,712,725,728,735,769,779,782,789,815,820,841,864,924,940,955,961,973]
results = {}
for pid in patient_ids:
    patient = Dosemetry(pid)
    patient.run_full_analysis()
    patient.get_firma()
    results[pid] = {
        'total_leq': patient.Total_Leq,
        'lex8': patient.Lex8,
        'total_hours_worked': patient.total_hours_worked,
        'firma': patient.firma,
    }
# Print results for all patients
for pid, res in results.items():
    print(f"Patient {pid}: Total Leq = {res['total_leq']:.3f}, Lex,8 = {res['lex8']:.3f}, Hours Worked = {res['total_hours_worked']:.3f}, Firma = {res['firma']}")

Patient 129: Total Leq = 67.365, Lex,8 = 67.948, Hours Worked = 9.149, Firma = Gebyrgribbenes Sparekasse
Patient 131: Total Leq = 66.039, Lex,8 = 65.520, Hours Worked = 7.099, Firma = Svindel og Humbug Bank ApS
Patient 133: Total Leq = 94.691, Lex,8 = 93.343, Hours Worked = 5.865, Firma = Haard Metal
Patient 142: Total Leq = 67.535, Lex,8 = 67.579, Hours Worked = 8.082, Firma = Gebyrgribbenes Sparekasse
Patient 188: Total Leq = 72.378, Lex,8 = 72.286, Hours Worked = 7.832, Firma = Gebyrgribbenes Sparekasse
Patient 228: Total Leq = 79.950, Lex,8 = 78.807, Hours Worked = 6.149, Firma = Vuggestuen Skrig og Skraal
Patient 242: Total Leq = 84.393, Lex,8 = 82.143, Hours Worked = 4.765, Firma = Vuggestuen Skrig og Skraal
Patient 254: Total Leq = 68.649, Lex,8 = 69.168, Hours Worked = 9.015, Firma = Gebyrgribbenes Sparekasse
Patient 268: Total Leq = 65.911, Lex,8 = 65.392, Hours Worked = 7.099, Firma = Svindel og Humbug Bank ApS
Patient 386: Total Leq = 71.566, Lex,8 = 71.334, Hours Worked = 7

Ud fra results dictionary kan vi aggregere l_ex,8 værdierne for hver firma og få middelværdien fra disse.
Pandas har en smart indbygget metode, groupby, som gør dette meget let.

Som forventet ser vi de højeste værdier hos Haard Metal (som man må forventer laver en del støj).

In [51]:
oversigt_tabel = pd.DataFrame(results).T
oversigt_tabel_middel = oversigt_tabel.groupby("firma")["lex8"].agg(["mean"])
print(oversigt_tabel_middel)

                                 mean
firma                                
Gebyrgribbenes Sparekasse   70.006625
Haard Metal                 91.010857
Svindel og Humbug Bank ApS    69.2145
Trykkeriet Klatmaling       84.820167
Vuggestuen Skrig og Skraal   80.29125
